In [1]:
from random import choice

## Data generators

### First-last harmony

In [2]:
def first_last_generate(n = 10, length = 10, grammatical = True, 
                        vowels = None, transparent = None):
    """ Generates a collection words following the rule of first-last harmony.
    
    * n (int): number of strings that need to be generated;
    * length (int): length of every one of the generated strings;
    * grammatical (bool): if set to True, the correctly harmonizing
                          forms are generated, and if set to False,
                          the disharmonic forms are produced;
    * vowels (list): list of vowels among which the first-last
                     agreement is established;
    * transparent (list): list of irrelevant elements.
    """
    
    # initialization and sanity check for the list of vowels
    if vowels is None:
        vowels = ["a", "o"]
    elif len(vowels) < 2:
        raise IndexError("The vowel system needs to contain at least two distinct vowels.")
    
    # initialization and sanity check for the list of transparent elements
    if transparent is None:
        transparent = ["x"]
    elif [i for i in vowels if i in transparent]:
        raise ValueError("Lists of harmonizing vowels and transparent elements cannot overlap.")
    
    # generate the required number of harmonic strings
    strings = []
    for i in range(n):
        new = choice(vowels)
        new += "".join([choice(vowels + transparent) for j in range(length - 2)])
        if grammatical:
            new += new[0]
        else:
            new += choice([i for i in vowels if i != new[0]])
            
        strings.append(new)

    return strings

In [3]:
print(first_last_generate(n = 5, grammatical = True))
print(first_last_generate(n = 5, grammatical = False))

['aaxxaoaooa', 'axxoaxaxxa', 'oaoaxaaaao', 'oaoxaoxoxo', 'oaxoaxaaao']
['oaooaaoxaa', 'ooxxxaaaaa', 'aooaaxaxxo', 'aaaoaaxooo', 'aoaxooxxoo']


### VC harmony

In [4]:
def vc_harmony_generate(n = 10, length = 10, grammatical = True,
                       vowels = None, consonants = None):
    """ Generates a collection words following the rule of vowel-consonant harmony.
    
    * n (int): number of strings that need to be generated;
    * length (int): length of every one of the generated strings;
    * grammatical (bool): if set to True, the correctly harmonizing
                          forms are generated, and if set to False,
                          the disharmonic forms are produced;
    * vowels (list): list of vowels among which the agreement
                     is established;
    * consonants (list): list of consonants among which the agreement
                         is established.
    """
    
    # initialization and sanity check for the list of vowels
    if vowels is None:
        vowels = ["a", "o"]
    elif len(vowels) < 2:
        raise IndexError("The vowel system needs to contain at least two distinct vowels.")
        
    # initialization and sanity check for the list of consonants
    if consonants is None:
        consonants = ["p", "b"]
    elif len(consonants) < 2:
        raise IndexError("The consonant system needs to contain at least two distinct consonants.")
    elif [i for i in vowels if i in consonants] or [i for i in consonants if i in vowels]:
        raise ValueError("Lists of harmonizing vowels and transparent elements cannot overlap.")
        
    # generate the required number of harmonic strings
    strings = []
    for i in range(n):
        v = choice(vowels)
        c = choice(consonants)
        new = "".join([choice([v, c]) for j in range(length)])
        
        # the ungrammatical forms are created by taking a random index
        # and rewriting it to the opposite vowel / consonant
        if not grammatical:
            ind = choice(range(length))
            if new[ind] == c:
                new = new[:ind] + choice([i for i in consonants if i != c]) + new[ind + 1:]
            else:
                new = new[:ind] + choice([i for i in vowels if i != v]) + new[ind + 1:]
                
        strings.append(new)
    
    return strings

In [5]:
print(vc_harmony_generate(n = 5, grammatical = True))
print(vc_harmony_generate(n = 5, grammatical = False))

['bobobboboo', 'oopppppooo', 'boobobbobo', 'poppooopop', 'ooobbbbbbb']
['aoboboobbo', 'opooboooob', 'baapppapaa', 'aboboooboo', 'aoobobbobo']


### Structure sensitive muti-tier harmony

#### 1. Preparing a class to encode structure sensitive rules

In [6]:
class SSRule(object):
    """ A generic template for a structure-sensititve rule. 
    
    * symbols (tuple): list of tier symbols relevant for the generalization;
    * target (str): a target character context of which is important;
    * right_context (str): a context in which a target character
                           is projected on the tier;
    * can_follow (tuple): a list of tier symbols that are allowed after
                         the target character is projected.
    """
    def __init__(self, symbols, target, right_context, can_follow):
        self.symbols = symbols
        self.target = target
        self.right_context = right_context
        self.can_follow = can_follow

    def is_grammatical(self, string):
        """ Checks if the given form follows a rule that is encoded.
        
        * string (str): a string well-formedness of which needs to be checked.
        """
        
        # first, we get rid of all irrelevant symbols (not symbols and contexts)
        string = "".join([i for i in string if i in list(self.symbols) + [self.right_context]])
        
        # second, we construct a tier of that strings
        tier = ""
        for i in range(len(string)):
            if string[i] in self.symbols:
                if string[i] == self.target and i < len(string) - 1 and\
                    string[i + 1] == self.right_context:
                    tier += self.target
                elif string[i] != self.target:
                    tier += string[i]

        # third, we check if that tier is well-formed
        for t in range(len(tier)):
            if tier[t] == self.target and t < len(tier) - 1 and\
                tier[t + 1] not in self.can_follow:
                return False
        return True

In [7]:
R1 = SSRule(symbols = ("o", "e", "a"), target = "o", right_context = "x", can_follow = ("a", "o"))
R1.is_grammatical("oxoeaee")

False

In [8]:
# prose: vowels "o", "e", and "a" are projected on the tier, but "o" can only be projected
#        if it is immediately followed by "x". On the tier, after "o" we can only observe
#        "a" or "o", i.e. projected "o" cannot be followed by "e" over the tier.
R1 = SSRule(symbols = ("o", "e", "a"), target = "o", right_context = "x", can_follow = ("a", "o"))

# the tier is "aa", "o" is not projected because it is not followed by "x"
print(R1.is_grammatical("baboa"), "expected True") 
# the tier is "aoa", "o" is projected and followed by "a" (allowed)
print(R1.is_grammatical("baboxa"), "expected True")
# the tier is "ae", "o" is not projected because it is not followed by "x"
print(R1.is_grammatical("baboe"), "expected True")
# the tier is "aoe", "o" is projected and followed by "e" (NOT allowed)
print(R1.is_grammatical("baboxe"), "expected False")


# similarly, encoding another rule to make sure that we are dealing with multiple tiers
R2 = SSRule(symbols = ("b", "p", "d"), target = "b", right_context = "y", can_follow = ("b", "p"))

True expected True
True expected True
True expected True
False expected False


#### 2. Writing a generator of a sequence grammatical wrt the rule
TODO: make this function a method in the SSRule class above.

In [9]:
def generate_rule_sequence(rule, length = 7, grammatical = True):
    """ This function generates a sequence of symbols (un)grammatical 
        with respect to the given rule.
        
    * rule (SSRule): a rule describing a structure sensitive dependency;
    * length (int): length of the generated sequence;
    * grammatical (bool): produces correct form when set to True, and 
                          makes a mistake when set to False.
    """
    
    # the generation of the well-formed sequence is done by a simplistic FSA
    sequence = ""
    state = 0
    for i in range(length):
        
        # State 0: the target was not observed
        if state == 0:
            sequence += choice(list(rule.symbols) + [rule.right_context])
            if sequence[-1] == rule.target:
                state = 1
                
        # State 1: the target was observed
        elif state == 1:
            sequence += choice(list(rule.symbols) + [rule.right_context])
            if sequence[-1] == rule.right_context:
                state = 2
            elif sequence[-1] != rule.target:
                state = 0
                
        # State 2: the right context was observed
        elif state == 2:
            sequence += choice(list(rule.can_follow) + [rule.right_context])
            if sequence[-1] in rule.can_follow and sequence[-1] != rule.target:
                state = 0
                
    # if the ungrammatical form is needed, a violating sequence is generated
    # and inserted into a random position within the sequence
    if not grammatical:
        violate = rule.target + rule.right_context +\
            choice([i for i in list(rule.symbols) if i not in rule.can_follow])
        index_violate = choice(range(length - 3))
        sequence = sequence[:index_violate] + violate + sequence[index_violate + 3:]
        
    return sequence

In [10]:
examples1 = [generate_rule_sequence(R1) for i in range(1000)]
v1 = all(map(R1.is_grammatical, examples1))
examples2 = [generate_rule_sequence(R2) for i in range(1000)]
v2 = all(map(R2.is_grammatical, examples2))
examples3 = [generate_rule_sequence(R1, grammatical = False) for i in range(1000)]
v3 = not any(map(R1.is_grammatical, examples3))
examples4 = [generate_rule_sequence(R2, grammatical = False) for i in range(1000)]
v4 = not any(map(R2.is_grammatical, examples4))

assert all([v1, v2, v3, v4]) == True
print("The correctness of the sequence generator is verified.")

The correctness of the sequence generator is verified.


#### 3. Intertwine
Helper function for the next module, for taking two sequences that are grammatical wrt different rules and intertwining them.

In [11]:
def intertwine(str1, str2, r = (0, 3)):
    """ Intertwines two strings: str1 and str2. At every step, it takes
    some characters from one string, and then some characters from another.
    oxxooxa
    * str1 (str): the first string;
    * str2 (str): the second string;
    * r (tuple[int, int]): min and max+1 symbols to be taken.
    """
    new_string = ""
    current = choice([1, 2])
    while str1 or str2:
        if current == 1:
            cut = choice(range(r[0], r[1]))
            if len(str1) < cut:
                new = str1[:]
            else:
                new = str1[:cut]
            new_string += new
            str1 = str1[len(new):]
            current = 2
        elif current == 2:
            cut = choice(range(r[0], r[1]))
            if len(str2) < cut:
                new = str2[:]
            else:
                new = str2[:cut]
            new_string += new
            str2 = str2[len(new):]
            current = 1
    return new_string

In [12]:
intertwine("abcdefg", "12345678")

'a123bcde4f5678g'

#### 4. Generator for the IMTSL harmony

In [13]:
def imtsl_harmony_generate(n = 10, length = 10, grammatical = True,
                       rule_1 = None, rule_2 = None):
    """ Generates a collection words following the given rules of the structure-
        Takessensitive dependencies that involve several tiers.
    
    * n (int): number of strings that need to be generated;
    * length (int): length of every one of the generated strings;
    * grammatical (bool): if set to True, the correctly harmonizing
                          forms are generated, and if set to False,
                          the disharmonic forms are produced;
    * rule_1 (SSRule): the first rule describing a long-distant structure-
                      sensitive dependency;
    * rule_2 (SSRule): the second rule describing a long-distant structure-
                      sensitive dependency.
    """
    
    # set the rules to R1 and R2 encoded above
    if rule_1 == None:
        rule_1 = R1
    if rule_2 == None:
        rule_2 = R2
    
    strings = []
    for i in range(n):
        # generate two tiers independently, and then intertwine them
        # WARNING: the tier alphabets of the two rules cannot overlap
        #          (required by both learner and generator)
        len_part_1 = length // 2
        len_part_2 = length - len_part_1

        part_1 = generate_rule_sequence(rule_1, len_part_1)
        part_2 = generate_rule_sequence(rule_2, len_part_2)

        if not grammatical:
            mistake = choice(["R1", "R2", "both"])
            if mistake == "R1":
                part_1 = generate_rule_sequence(rule_1, len_part_1, grammatical = False)
            elif mistake == "R2":
                part_2 = generate_rule_sequence(rule_2, len_part_2, grammatical = False)
            else:
                part_1 = generate_rule_sequence(rule_1, len_part_1, grammatical = False)
                part_2 = generate_rule_sequence(rule_2, len_part_2, grammatical = False)

        # intertwining the two generated sequences
        new_string = intertwine(part_1, part_2)
        strings.append(new_string)
    return strings

In [14]:
print(imtsl_harmony_generate(n = 15, length = 15))

['yyaxbaeboyyoepy', 'eoyyyppoopaodye', 'eaeodpaayoybbyp', 'oeabbaeexyypypb', 'yyxobpxyaodypeo', 'ydpexyaxebdyoob', 'xoydxaeexddbyyb', 'eopoeoaeyddpdpd', 'xabbdeydeeyobya', 'xxppbxodbaexpdd', 'bxppbdaxxeppbaa', 'odapdoxaoxpbypy', 'edobyyyeapxyyex', 'abbaxxxdddbaypx', 'dpaopboppbyooex']


In [15]:
good = imtsl_harmony_generate(n = 1000, length = 15)
good_evals = list(map(all, zip(map(R1.is_grammatical, good), map(R2.is_grammatical, good))))
bad = imtsl_harmony_generate(n = 1000, length = 15, grammatical = False)
bad_evals = list(map(all, zip(map(R1.is_grammatical, bad), map(R2.is_grammatical, bad))))
assert all(good_evals) == (not any(bad_evals)) == True
print("The correctness of the generator is verified.")

The correctness of the generator is verified.


## Code of the learning algorithm